In [ ]:
import BAC0
import time
import os, sys
import paho.mqtt.client as mqtt
from random import randrange, uniform
import uuid

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [ ]:
def get_bac0_device(device, network):
    print("Get BAC0 device")
    name, vendor, address, device_id = device
    return BAC0.device(address, device_id, network, poll=0,
    object_list=None)

In [ ]:
def main():
    # MQTT Setup
    
    # Local or azure mqtt broker address
    mqttBroker ="127.0.0.1"
    # username = "mqtt_username"
    # password = "mqtt_password"
    port = 1883
    cid = str(uuid.uuid4())
    client = mqtt.Client("Gateway_Device_" + cid)
    # client.username_pw_set(username, password)
    
    #Connect to MQTT Broker
    client.connect(mqttBroker, port)

    # BAC0 Setup
    bacnet = BAC0.connect()
    bacnet.discover()
    device = bacnet.devices[0]
    bac0_device = get_bac0_device(device, bacnet)
    
    # Main Loop of sending data
    while True:
        points = bac0_device.points
        blockPrint()
        print(points)
        enablePrint()
        for idx, point in enumerate(points):
            msg = str(point.lastValue)
            topic = "{}".format(point.properties.name)
            client.publish(topic, msg)
        
        # Wait for 4 seconds before sending updated data
        time.sleep(4)

In [ ]:
if __name__ == "__main__":
    main()